In [1]:
import pandas as pd
import numpy as np


In [2]:
raw_data = pd.read_csv("data.csv")
raw_data.shape

(1467832, 63)

In [3]:
platform_dummies = pd.get_dummies(raw_data['platform'], prefix = 'platform')
media_dummies = pd.get_dummies(raw_data['media_source'], prefix = '')
country_dummies = pd.get_dummies(raw_data['country_code'], prefix = '')

In [4]:
raw_data = pd.concat([raw_data, platform_dummies, media_dummies, country_dummies], axis = 1)
dummy_data = raw_data
dummy_data.shape

(1467832, 313)

In [5]:
raw_data.drop('platform', axis = 1, inplace = True)
raw_data.drop('country_code', axis = 1, inplace = True)
raw_data.drop('media_source', axis = 1, inplace = True)



raw_data.shape

(1467832, 310)

In [6]:
from datetime import datetime
"""
Data needed to be transformed into:
1) Time since 1970 (Unix date) continuous
2) Day of a year [1, 365] categorical
3) Month of a year [1, 12] categorical
4) Day of a week in [1, 7] categorical
5) Is weekend binary
"""


def transform_date(dates_frame: pd.DataFrame): # return pd.DataFrame
    
    py_dates = []
    for el in dates_frame:
        py_dates.append(datetime.strptime(el, '%Y-%m-%d'))

    
    dates_list = [] # Unix time 
    day_of_a_year_list = [] # Day of year
    month_of_a_year = [] # month of a year
    day_of_a_week = [] # day of a week
    is_weekend = [] # is day a weekend 


    for el in py_dates:
        dates_list.append(el.strftime("%s"))
        day_of_a_year_list.append(el.timetuple().tm_yday)
        month_of_a_year.append(el.timetuple().tm_mon)
        day_of_a_week.append(el.weekday())
        is_weekend.append((el.weekday() > 4))
    
    
    a = pd.DataFrame({"_":day_of_a_year_list})
    b = pd.DataFrame({"_":day_of_a_week})
    c = pd.DataFrame({"_": month_of_a_year})
                     
    d_year_day = pd.get_dummies(a["_"], prefix = 'day')
    d_month_of_year = pd.get_dummies(c["_"], prefix = 'month')
    d_weekday = pd.get_dummies(b["_"], prefix = 'weekday')
    d_weekend = pd.DataFrame({"is_weekend" : is_weekend})

    print(d_year_day.shape, d_month_of_year.shape, d_weekday.shape, d_weekend.shape, sep = '\n')
    
    return pd.concat([pd.DataFrame({"unix date": dates_list}),
                        d_year_day, d_month_of_year, d_weekday, d_weekend], axis = 1)



In [7]:
# To much features 
# dummy_data = pd.concat([dummy_data,transform_date(raw_data['install_date'])], axis = 1)
dummy_data.drop('install_date', axis = 1, inplace=True)
dummy_data.shape

# Now dataframe is fully prepeared for machine learning

(1467832, 309)

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [9]:
# Spliting dataset for training 
train, test = train_test_split(dummy_data, test_size=0.0001)

train_set_y = train.drop(train.columns.difference(['target_sub_ltv_day30', 'target_iap_ltv_day30', 'target_ad_ltv_day30',
       'target_full_ltv_day30']), 1) 

train.drop(['target_sub_ltv_day30', 'target_iap_ltv_day30', 'target_ad_ltv_day30',
       'target_full_ltv_day30'], axis = 1, inplace = True)


test_set_y = test.drop(train.columns.difference(['target_sub_ltv_day30', 'target_iap_ltv_day30', 'target_ad_ltv_day30',
       'target_full_ltv_day30']), 1) 

test.drop(['target_sub_ltv_day30', 'target_iap_ltv_day30', 'target_ad_ltv_day30',
       'target_full_ltv_day30'], axis = 1, inplace = True)

# just synonims

train_set_x = train
test_set_x = test

train_set_x.shape
train_set_y.shape
test_set_x.shape
test_set_y.shape


<ipython-input-9-8439f0fa05a0>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train_set_y = train.drop(train.columns.difference(['target_sub_ltv_day30', 'target_iap_ltv_day30', 'target_ad_ltv_day30',
<ipython-input-9-8439f0fa05a0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['target_sub_ltv_day30', 'target_iap_ltv_day30', 'target_ad_ltv_day30',
<ipython-input-9-8439f0fa05a0>:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  test_set_y = test.drop(train.columns.difference(['target_sub_ltv_day30', 'target_iap_ltv_day30', 'target_ad_ltv_day30',
<ipython-input-9-8439f0fa05a0>:14: SettingWithCopyWarning: 
A value is 

(147, 4)

In [10]:
#reg = LinearRegression().fit(train_set_x, train_set_y)
#reg.score(train_set_x, train_set_y)

# such colums as 'total_sessions_day0', 'total_sessions_day1', 'total_sessions_day3', 'total_sessions_day7' 
# can contain nan values 

train_set_x.fillna(0)
test_set_x.fillna(0)

train_set_y.fillna(0)
test_set_y.fillna(0)

,target_sub_ltv_day30,target_iap_ltv_day30,target_ad_ltv_day30,target_full_ltv_day30
678859,0.0,0.000000,0.000000,0.000000
565814,0.0,0.000000,0.077709,0.077709
1078906,0.0,0.997284,0.000000,0.997284
390092,0.0,8.773434,0.139739,8.913173
1045579,0.0,0.000000,0.000000,0.000000
...,...,...,...,...
904457,0.0,0.000000,0.013776,0.013776
299756,0.0,0.000000,0.000000,0.000000
70554,0.0,0.000000,0.000000,0.000000
295899,0.0,0.000000,0.000000,0.000000


In [13]:
import tensorflow as tf

train_set_x = train_set_x.reset_index()
train_set_y = train_set_y.reset_index()

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_set_x))

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=train_set_x.shape[1]),
    tf.keras.layers.Dense(units=1)
])

#model.summary()


ValueError: cannot insert level_0, already exists

In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [16]:
model.fit(
    train_set_y,
    train_set_y,
    epochs=100,
    # Suppress logging.
    verbose=1,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

Epoch 1/100
36693/36693 [==============================] - 77s 2ms/step - loss: 303036.8125 - val_loss: 342497.6875
Epoch 2/100
36693/36693 [==============================] - 77s 2ms/step - loss: 281964.5625 - val_loss: 342473.9062
Epoch 3/100
36693/36693 [==============================] - 78s 2ms/step - loss: 285982.0312 - val_loss: 380013.3125
Epoch 4/100
36693/36693 [==============================] - 77s 2ms/step - loss: 295409.9375 - val_loss: 343611.7188
Epoch 5/100
36693/36693 [==============================] - 77s 2ms/step - loss: 292855.1562 - val_loss: 457013.9375
Epoch 6/100
36693/36693 [==============================] - 77s 2ms/step - loss: 289146.9688 - val_loss: 342488.5000
Epoch 7/100
36693/36693 [==============================] - 77s 2ms/step - loss: 301873.5000 - val_loss: 344310.7500
Epoch 8/100
36693/36693 [==============================] - 77s 2ms/step - loss: 283364.1250 - val_loss: 406188.9688
Epoch 9/100
36693/36693 [==============================] - 76s 2ms/step 

KeyboardInterrupt: 